In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
from api_keys import weather_api_key

from citipy import citipy

In [24]:
BASE_URL = "https://api.geoapify.com/v2/places"
HTTP_STATUS_OK = 200
HTTP_STATUS_NOT_FOUND = 404

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chonchi,-42.6128,-73.8092,16.73,53,21,1.72,CL,1707778011
1,1,albany,42.6001,-73.9662,3.23,67,100,0.45,US,1707777803
2,2,blackmans bay,-43.0167,147.3167,20.96,64,83,0.45,AU,1707778012
3,3,selfoss,63.9331,-20.9971,-5.26,89,99,4.89,IS,1707778013
4,4,kodiak,57.7900,-152.4072,4.73,87,100,6.17,US,1707778013


In [3]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[
    (city_data_df["Wind Speed"] <= 5) & (city_data_df["Humidity"] < 50) &(city_data_df["Cloudiness"] < 50) & (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 40)
]

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,new norfolk,-42.7826,147.0587,21.62,28,23,1.55,AU,1707778036
95,95,adrar,20.5022,-10.0711,23.62,26,0,4.04,MR,1707778068
103,103,kassala,15.4510,36.4000,24.20,34,4,3.53,SD,1707778074
143,143,biltine,14.5333,20.9167,22.57,10,0,4.34,TD,1707778108
145,145,jwaneng,-24.6004,24.7303,22.03,43,32,4.42,BW,1707778110
170,170,santa ana,33.7456,-117.8678,22.10,42,0,4.63,US,1707778065
248,248,ciudad insurgentes,25.2625,-111.7753,23.73,16,6,3.07,MX,1707778180
320,320,tidjikja,18.5564,-11.4271,26.81,26,0,4.27,MR,1707778242
322,322,bairnsdale,-37.8333,147.6167,32.81,42,1,2.68,AU,1707778243
328,328,freire,-38.9500,-72.6333,26.89,39,0,4.12,CL,1707778247


In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copy_ideal_weather = ideal_weather.copy()
hotel_df = copy_ideal_weather[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

C:\Users\macol\AppData\Local\Temp\ipykernel_16416\3675416011.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
45,new norfolk,AU,-42.7826,147.0587,28,
95,adrar,MR,20.5022,-10.0711,26,
103,kassala,SD,15.4510,36.4000,34,
143,biltine,TD,14.5333,20.9167,10,
145,jwaneng,BW,-24.6004,24.7303,43,
170,santa ana,US,33.7456,-117.8678,42,
248,ciudad insurgentes,MX,25.2625,-111.7753,16,
320,tidjikja,MR,18.5564,-11.4271,26,
322,bairnsdale,AU,-37.8333,147.6167,42,
328,freire,CL,-38.9500,-72.6333,39,


In [ ]:
# test = 'https://api.geoapify.com/v2/places?categories=public_transport.subway&
#filter=circle:-0.07071648508463113,51.50848194136378,1000&bias=proximity:-0.07071648508463113,51.50848194136378&
#limit=20&apiKey=7aa85fddfbfe4c16994e21e24a66645f'
# next_test = requests.get(test).json()
params


In [5]:
test_hotel_list = ['london','new york']

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit=20
categories="accommodation.hotel"

params = {
    "categories": categories,
    "radius": radius,
    "limit": limit,
    "apiKey": geoapify_key
}


In [ ]:
# Print a message to follow up the hotel search
print("Starting hotel search")

In [ ]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

In [ ]:
name_address = requests.get(base_url, params=params)
name_address

In [7]:
test_cities = ["london","miami","berlin"]

In [8]:
# Set the API base URL
url = "https://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []

In [9]:
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(test_cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}units=metric&q={city}&appid={weather_api_key}"
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                         })

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | london
Processing Record 2 of Set 1 | miami
Processing Record 3 of Set 1 | berlin
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng
0,london,51.5085,-0.1257
1,miami,25.7743,-80.1937
2,berlin,52.5244,13.4105


In [11]:
city_data_df["Hotel Name"] = ""
city_data_df

,City,Lat,Lng,Hotel Name
0,london,51.5085,-0.1257,
1,miami,25.7743,-80.1937,
2,berlin,52.5244,13.4105,


In [40]:
# Set parameters to search for a hotel
radius = 10000
limit=20
categories="accommodation.hotel"

params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in city_data_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = city_data_df.loc[index, "Lat"]
    longitude = city_data_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Make and API request using the params dictionaty
    response = requests.get(BASE_URL, params=params)
    status_code = response.status_code
    if status_code == HTTP_STATUS_OK:      
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            city_data_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            city_data_df.loc[index, "Hotel Name"] = "No hotel found"

        # Log the search results
        print(f"{city_data_df.loc[index, 'City']} - nearest hotel: {city_data_df.loc[index, 'Hotel Name']}")
    elif status_code == HTTP_STATUS_NOT_FOUND:
        # If no hotel is found, set the hotel name as "No hotel found".
        city_data_df.loc[index, "Hotel Name"] = "No hotel found"
        print(f"No hotel found for {city}. url:{response.url}")
    else:
        msg = f"Invalid status code {status_code} for url {response.url}"
        raise ValueError(msg)
# Display sample data
city_data_df

Starting hotel search
london - nearest hotel: The Clermont
miami - nearest hotel: The Langford
berlin - nearest hotel: Lux 11


,City,Lat,Lng,Hotel Name
0,london,51.5085,-0.1257,The Clermont
1,miami,25.7743,-80.1937,The Langford
2,berlin,52.5244,13.4105,Lux 11


In [27]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Lux 11',
    'country': 'Germany',
    'country_code': 'de',
    'city': 'Berlin',
    'postcode': '10178',
    'district': 'Mitte',
    'neighbourhood': 'Scheunenviertel',
    'suburb': 'Mitte',
    'street': 'Rosa-Luxemburg-Straße',
    'housenumber': '11-13',
    'lon': 13.4098329,
    'lat': 52.5245961,
    'formatted': 'Lux 11, Rosa-Luxemburg-Straße 11-13, 10178 Berlin, Germany',
    'address_line1': 'Lux 11',
    'address_line2': 'Rosa-Luxemburg-Straße 11-13, 10178 Berlin, Germany',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': ['details',
     'details.accommodation',
     'details.contact',
     'details.facilities'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Lux 11',
      'rooms': 72

In [23]:
print(name_address["type"][])

e


In [39]:
name_address["features"][0]["properties"]["name"]

'Lux 11'

In [35]:
name_address["features"][0].keys()

dict_keys(['type', 'properties', 'geometry'])